# Financial Analysis System with RagaAI Catalyst Integration

This Jupyter notebook demonstrates the integration of Catalyst, a powerful tracing and monitoring tool, with a financial analysis system. Catalyst provides seamless tracing capabilities for both function calls and AI model interactions, allowing for comprehensive analysis and debugging of complex systems.

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Step 1: Import RagaAI Catalyst components for tracing and monitoring
from ragaai_catalyst.tracers import Tracer
from ragaai_catalyst import (
    RagaAICatalyst,
    trace_agent,
    trace_llm,
    trace_tool,
    init_tracing,
)
from textblob import TextBlob
from dotenv import load_dotenv
import os
from openai import OpenAI
import random

In [2]:
# Step 2: Load environment variables for RagaAI credentials
load_dotenv()

True

In [3]:
# Step 3: Initialize RagaAI Catalyst with authentication details
catalyst = RagaAICatalyst(
    access_key=os.getenv("RAGAAI_CATALYST_ACCESS_KEY"),
    secret_key=os.getenv("RAGAAI_CATALYST_SECRET_KEY"),
    base_url=os.getenv("RAGAAI_CATALYST_BASE_URL"),
)

Token(s) set successfully


## FinancialAnalysisSystem Class

Now, let's define our `FinancialAnalysisSystem` class with Catalyst integration.

In [5]:
# Step 4: Configure tracer for Financial Analysis
tracer = Tracer(
    project_name="tracer",
    dataset_name="testing",
    tracer_type="anything",
    metadata={"model": "gpt-4o-mini", "environment": "production"},
    pipeline={
        "llm_model": "gpt-4o-mini",
        "vector_store": "faiss",
        "embed_model": "text-embedding-ada-002",
    },
)
# Step 5: Initialize the tracing system
init_tracing(catalyst=catalyst, tracer=tracer)

In [6]:
class FinancialAnalysisSystem:
    def __init__(self):
        self.stock_data = {}
        self.news_sentiment = {}
        self.economic_indicators = {}

    # We can trace the tools using the `@trace_tool` decorator
    # Step 6: Trace all the tools using the `@trace_tool` decorator
    # Use the `trace_tool` decorator to trace the `fetch_stock_data` method
    @trace_tool(name="fetch_stock_data")
    def fetch_stock_data(self, symbol):
        return {
            "symbol": symbol,
            "price": round(random.uniform(50, 500), 2),
            "change": round(random.uniform(-5, 5), 2),
        }

    # Use the `trace_tool` decorator to trace the `fetch_news_articles` method
    @trace_tool(name="fetch_news_articles")
    def fetch_news_articles(self, company):
        return [
            f"{company} announces new product line",
            f"{company} reports quarterly earnings",
            f"{company} faces regulatory scrutiny",
        ]

    # Tracing the `analyze_sentiment` method using the `trace_tool` decorator
    @trace_tool(name="analyze_sentiment")
    def analyze_sentiment(self, text):
        return TextBlob(text).sentiment.polarity

    # Tracing the `fetch_economic_indicators` method using the `trace_tool` decorator
    @trace_tool(name="fetch_economic_indicators")
    def fetch_economic_indicators(self):
        return {
            "gdp_growth": round(random.uniform(-2, 5), 2),
            "unemployment_rate": round(random.uniform(3, 10), 2),
            "inflation_rate": round(random.uniform(0, 5), 2),
        }

    # We can trace the LLMs using the `@trace_llm` decorator
    # Step 7: Trace all the LLMs using the `@trace_llm` decorator
    # Tracing the `analyze_market_conditions` method using the `trace_llm` decorator
    @trace_llm(name="analyze_market_conditions")
    def analyze_market_conditions(self, stock_data, sentiment, economic_indicators):
        prompt = f"""
        Analyze the following market conditions and provide a brief market outlook:
        Stock: {stock_data["symbol"]} at ${stock_data["price"]} (change: {stock_data["change"]}%)
        News Sentiment: {sentiment}
        Economic Indicators:
        - GDP Growth: {economic_indicators["gdp_growth"]}%
        - Unemployment Rate: {economic_indicators["unemployment_rate"]}%
        - Inflation Rate: {economic_indicators["inflation_rate"]}%
        """
        client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        response = client.chat.completions.create(
            model="gpt-4-0125-preview",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=150,
        )
        return response.choices[0].message.content.strip()

    # Tracing the `generate_investment_recommendation` method using the `trace_llm` decorator
    @trace_llm(name="generate_investment_recommendation")
    def generate_investment_recommendation(self, market_outlook, risk_tolerance):
        prompt = f"""
        Based on the following market outlook and investor risk tolerance,
        provide a specific investment recommendation:
        Market Outlook: {market_outlook}
        Investor Risk Tolerance: {risk_tolerance}
        """
        client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        response = client.chat.completions.create(
            model="gpt-4-0125-preview",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=200,
        )
        return response.choices[0].message.content.strip()

    # We can trace the agents using the `@trace_agent` decorator
    # Step 8: Trace all the agents using the `@trace_agent` decorator
    # Tracing the `financial_advisor_agent` agent using the `trace_agent` decorator
    @trace_agent(name="FinancialAdvisorAgent")
    def financial_advisor_agent(self, stock_symbol, risk_tolerance):
        self.stock_data = self.fetch_stock_data(stock_symbol)
        news_articles = self.fetch_news_articles(stock_symbol)
        sentiment_scores = [
            self.analyze_sentiment(article) for article in news_articles
        ]
        self.news_sentiment = sum(sentiment_scores) / len(sentiment_scores)
        self.economic_indicators = self.fetch_economic_indicators()
        market_outlook = self.analyze_market_conditions(
            self.stock_data, self.news_sentiment, self.economic_indicators
        )
        recommendation = self.generate_investment_recommendation(
            market_outlook, risk_tolerance
        )
        return recommendation

    def run_analysis(self, stock_symbol, risk_tolerance):
        recommendation = self.financial_advisor_agent(stock_symbol, risk_tolerance)
        print(f"\nAnalysis for {stock_symbol}:")
        print(f"Stock Data: {self.stock_data}")
        print(f"News Sentiment: {self.news_sentiment}")
        print(f"Economic Indicators: {self.economic_indicators}")
        print(f"\nInvestment Recommendation:\n{recommendation}")
        if "buy" in recommendation.lower():
            self.execute_buy_order(stock_symbol)
        elif "sell" in recommendation.lower():
            self.execute_sell_order(stock_symbol)
        else:
            print("No action taken based on the current recommendation.")

    @trace_tool(name="execute_buy_order")
    def execute_buy_order(self, symbol):
        print(f"Executing buy order for {symbol}")

    @trace_tool(name="execute_sell_order")
    def execute_sell_order(self, symbol):
        print(f"Executing sell order for {symbol}")

## Running the Analysis

Now let's create an instance of our `FinancialAnalysisSystem` and run an analysis.

In [ ]:
# Step 9: Run the financial analysis system with tracer
with tracer:
    # Create an instance of FinancialAnalysisSystem
    analysis_system = FinancialAnalysisSystem()
    # Run an analysis for Apple stock with moderate risk tolerance
    analysis_system.run_analysis("AAPL", "moderate")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:ragaai_catalyst.tracers.agentic_tracing.utils.zip_list_of_unique_files: Zip file created successfully.
INFO:ragaai_catalyst.tracers.agentic_tracing.tracers.base: Traces saved successfully.



Analysis for AAPL:
Stock Data: {'symbol': 'AAPL', 'price': 458.0, 'change': 2.91}
News Sentiment: 0.04545454545454545
Economic Indicators: {'gdp_growth': 1.85, 'unemployment_rate': 6.85, 'inflation_rate': 0.06}

Investment Recommendation:
Based on the provided market outlook and the investor's moderate risk tolerance, I would recommend considering a balanced approach to investing in Apple Inc. (AAPL) stock. Here's my reasoning and recommendation:

### Rationale:

1. **Stock Performance**: Apple has recently shown a positive increase of 2.91% in its stock price, which can be considered a good sign of its current market performance and strength. This uptick could signal a healthy financial outlook or a response to favorable news about the company, indicating its potential for continued growth.

2. **News Sentiment**: The news sentiment score of 0.04545454545454545, although marginally positive, is quite close to neutral. This suggests that while there might be more positive news than ne